# Работа с различными типами признаков
В первом спринте вы уже проводили анализ вашего датасета. Тогда вы изучили типы данных, которые присутствовали в выборке, и проанализировали каждый тип признаков. Тот опыт был необходим — это основа работы с данными. Теперь, когда вы на этапе подготовки модели к продакшену, пора расширить ваши знания в нескольких областях.
В реальной жизни данные зачастую разнообразны и неоднородны. Перед началом обучения модели нужно выполнить предварительную обработку (англ. preprocessing) — она касается различных типов признаков: числовых, текстовых, дат и других. В этом уроке мы расскажем о методах обработки.
Один из ключевых этапов уже после предобработки данных — конструирование признаков (англ. feature engineering). Облегчат этот процесс несколько существующих и уже знакомых вам инструментов: Pipeline и ColumnTransformer из библиотеки scikit-learn. В ходе урока мы затронем их ещё раз. 
Для успешной предварительной обработки сначала нужно отделить данные разных типов друг от друга. Для этой задачи у датафрейма из библиотеки pandas есть метод select_dtypes, который позволяет включать (параметр include) или исключать (параметр exclude) определённые типы данных. Вот некоторые из типов данных:
для выбора всех числовых типов используют np.number или 'number';
для выбора чисел с плавающей запятой используют ‘float’ или, например, ‘float64’;
для выбора строковых типов данных используют object dtype, однако помните, что это вернёт все столбцы с object dtype — даже категориальные признаки;
Для выбора дат используют np.datetime64, 'datetime' или 'datetime64'.
Полный список названий типов базируется на библиотеке NumPy, c их иерархией можно ознакомиться в документации на официальном сайте NumPy.

### Задание 1
В коде ниже мы генерируем несколько колонок с данными и объединяем их в единый датафрейм, сохранённый в переменную df. Ваша задача — прописать информацию о типах данных, содержащихся в переменной df, а также создать переменные df_int, df_float, df_bool, df_object, df_date, куда сохранятся наборы данных, соответствующие этим типам.

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

np.random.seed(42)
np.random.default_rng(42)

# генерация данных для каждого столбца
data = {
    'temperature_celsius': np.random.uniform(20, 35, size=100),  # температура в градусах Цельсия (float)
    'age_years': np.random.randint(18, 65, size=100),  # возраст в годах (int)
    'timestamp_event': [pd.Timestamp('20230101') + timedelta(days=i) for i in range(100)],  # время события (datetime)
    'product_category': np.random.choice(['electronics', 'clothing', 'food'], size=100),  # категория продукта (string)
    'is_purchased': np.random.choice([True, False], size=100),  # булевое значение приобретения (bool)
    'humidity_percentage': np.random.uniform(40, 80, size=100),  # влажность в процентах (float)
    'income_usd': np.random.randint(20000, 100000, size=100),  # доход в долларах США (int)
    'last_updated': [pd.Timestamp('20240101') + timedelta(days=i) for i in range(100)],  # последнее обновление (datetime)
    'product_name': ['Product_' + str(i) for i in range(100)],  # название продукта (string)
    'is_subscribed': np.random.choice([True, False], size=100)  # булевое значение подписки (bool)
}

# создание DataFrame
df = pd.DataFrame(data)

# Информация о типах данных
print("Типы данных в датафрейме:")
print(df.dtypes)
print("\n" + "="*50 + "\n")

# Выбор данных по типам
df_int = df.select_dtypes(include='int')
df_float = df.select_dtypes(include='float')
df_bool = df.select_dtypes(include='bool')
df_object = df.select_dtypes(include='object')
df_date = df.select_dtypes(include='datetime')

# Проверка результатов
print("Колонки типа int:")
print(df_int.columns.tolist())
print(f"\nКоличество колонок: {len(df_int.columns)}\n")

print("Колонки типа float:")
print(df_float.columns.tolist())
print(f"\nКоличество колонок: {len(df_float.columns)}\n")

print("Колонки типа bool:")
print(df_bool.columns.tolist())
print(f"\nКоличество колонок: {len(df_bool.columns)}\n")

print("Колонки типа object:")
print(df_object.columns.tolist())
print(f"\nКоличество колонок: {len(df_object.columns)}\n")

print("Колонки типа datetime:")
print(df_date.columns.tolist())
print(f"\nКоличество колонок: {len(df_date.columns)}")

Типы данных в датафрейме:
temperature_celsius           float64
age_years                       int64
timestamp_event        datetime64[ns]
product_category               object
is_purchased                     bool
humidity_percentage           float64
income_usd                      int64
last_updated           datetime64[ns]
product_name                   object
is_subscribed                    bool
dtype: object


Колонки типа int:
['age_years', 'income_usd']

Количество колонок: 2

Колонки типа float:
['temperature_celsius', 'humidity_percentage']

Количество колонок: 2

Колонки типа bool:
['is_purchased', 'is_subscribed']

Количество колонок: 2

Колонки типа object:
['product_category', 'product_name']

Количество колонок: 2

Колонки типа datetime:
['timestamp_event', 'last_updated']

Количество колонок: 2


Сейчас, когда вы научились отбирать нужные типы данных, рассмотрите способы их эффективной обработки. За это отвечает модуль preprocessing в библиотеке scikit-learn. Он предоставляет широкий спектр инструментов, которые помогут вам не только преобразовать данные, но и подготовить их к использованию в моделях машинного обучения.
Для чего эти инструменты будут полезны:
Для кодирования категориальных переменных в числовые — это позволит моделям работать с данными, которые содержат текстовую или категориальную информацию;
Для нормализации данных — так улучшится их интерпретируемость для модели;
Для заполнения пропущенных значений;
Для масштабирования числовых признаков, чтобы модель справилась с данными в различных диапазонах.
Ниже приведён полный список доступных объектов и методов предобработки данных в модуле preprocessing из библиотеки scikit-learn:

Подробнее про них вы можете прочитать в документации на официальном сайте sklearn.
Каждый объект предобработки данных обладает набором методов, который делает его универсальным для работы с данными различных типов. Вот основные методы:
fit — оценка параметров модели на основе данных X,
transform — преобразование данных X,
fit_transform — поочерёдное применение методов выше к данным X.
Также есть несколько дополнительных: 
get_feature_names_out — получение имён выходных признаков после преобразования,
get_params — получение параметров модели. Метод, который возвращает параметры объекта, установленные во время инициализации,
inverse_transform — обратное преобразование данных X. Метод, который использует ранее рассчитанные параметры, чтобы вернуть данные к исходному масштабу,
set_params — установка параметров модели. Этот метод позволяет задать параметры модели после её инициализации.
Чтобы ознакомиться с ними подробнее, загляните на официальный сайт sklearn.
Основные методы можно применить так:

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np

X_train = np.array([[ 1., -1.,  2.],
[ 2.,  0.,  0.],
[ 0.,  1., -1.]])

transformer = StandardScaler().fit(X_train) # оцениваем параметры модели
transformer.mean_ # получаем средние значения для преобразования
transformer.scale_ # получаем масштабы для преобразования

X_scaled = transformer.transform(X_train) # применяем преобразование к данным
X_scaled # получаем преобразованные данные

Посмотрите на другие популярные преобразования:
OneHotEncoder (унитарное кодирование)
  Этот энкодер преобразует категориальные (дискретные) признаки в числовые, создавая бинарные столбцы для каждой категории. В качестве входных данных для этого преобразования должен выступать массив целых чисел или строк, чьи значения принимаются как категориальные признаки. Энкодер создаёт разреженную матрицу или массив (в зависимости от параметра sparse_output). Это кодирование необходимо для подачи категориальных данных во многие оценщики scikit-learn, особенно в линейные модели и SVM (метод опорных векторов) со стандартными ядрами.
  Пример использования: если у вас есть признак «Тип автомобиля» с категориями «Седан», «Хэтчбек», «Универсал», то OneHotEncoder поможет преобразовать его в бинарные столбцы, чтобы использовать в моделях машинного обучения.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

data = np.array(['Универсал', 'Седан', 'Универсал', 'Хэтчбек']).reshape(-1, 1)
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(data).toarray()  


SplineTransformer (преобразование сплайнов)
  Этот энкодер создаёт новую матрицу признаков, состоящую из сплайнов порядка degree. Количество сгенерированных сплайнов равно n_splines=n_knots + degree - 1 для каждого признака, где
n_knots определяет количество узлов (точек, в которых сопрягаются сплайны) для каждого признака. Он указывает, сколько понадобится точек, чтобы разбить признак на интервалы. В этих интервалах сплайны будут аппроксимировать данные.
degree определяет порядок полинома, используемого для построения сплайнов. Это число указывает степень полинома, применяемого для каждого сплайна.
Пример сплайна:

   Пример использования: в финансовом анализе для оценки временных рядов (например, прогнозирования курсов валют) — с помощью сплайнов можно генерировать гладкие кривые, чтобы проследить изменения данных во времени.

In [ ]:
from sklearn.preprocessing import SplineTransformer
import numpy as np
    
X = np.random.rand(10, 1)
transformer = SplineTransformer(n_knots=3, degree=2)
X_transformed = transformer.fit_transform(X)    

QuantileTransformer (квантильное преобразование)
  Этот метод преобразует признаки, чтобы они распределялись равномерно или нормально — так данные меньше подвергаются влиянию выбросов. Преобразование применяется к каждому признаку независимо. Идея метода такова: оценить функцию распределения признака, чтобы преобразовать исходные значения в равномерное распределение. 
  Пример использования: если у вас есть данные о доходах с широким диапазоном значений, квантильное преобразование сделает их более сопоставимыми и устойчивыми к выбросам.

In [ ]:
  from sklearn.preprocessing import QuantileTransformer
  import numpy as np
  
  X = np.random.rand(10, 1)
  transformer = QuantileTransformer()
  X_transformed = transformer.fit_transform(X)
  

KBinsDiscretizer (дискретизация признаков)
  Этот энкодер разбивает непрерывные данные на интервалы.
  Пример использования: если вам нужно проанализировать рынок потребления товаров в зависимости от возраста покупателей, то KBinsDiscretizer поможет с группировкой возрастных данных по группам. 

In [ ]:
  from sklearn.preprocessing import KBinsDiscretizer
  import numpy as np
  
  X = np.array([[2.3], [5.6], [7.8], [1.2]])
  est = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
  est.fit(X)
  transformed = est.transform(X)


### Задание 2
Примените преобразования к вашим данным:
Binarizer — к колонке income_usd (в качестве границы возьмите средние значения). Результат сохраните в колонку income_usd_binarized.
StandardScaler — к колонке age_years. Результат сохраните в колонку age_years_standarded.
LabelEncoder — к колонке is_subscribed. Результат сохраните в колонку is_subscribed_encoded.

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.preprocessing import Binarizer, StandardScaler, LabelEncoder

np.random.seed(42)
np.random.default_rng(42)

# генерация данных для каждого столбца
data = {
    'temperature_celsius': np.random.uniform(20, 35, size=100),  # температура в градусах Цельсия (float)
    'age_years': np.random.randint(18, 65, size=100),  # возраст в годах (int)
    'timestamp_event': [pd.Timestamp('20230101') + timedelta(days=i) for i in range(100)],  # время события (datetime)
    'product_category': np.random.choice(['electronics', 'clothing', 'food'], size=100),  # категория продукта (string)
    'is_purchased': np.random.choice([True, False], size=100),  # булевое значение приобретения (bool)
    'humidity_percentage': np.random.uniform(40, 80, size=100),  # влажность в процентах (float)
    'income_usd': np.random.randint(20000, 100000, size=100),  # доход в долларах США (int)
    'last_updated': [pd.Timestamp('20240101') + timedelta(days=i) for i in range(100)],  # последнее обновление (datetime)
    'product_name': ['Product_' + str(i) for i in range(100)],  # название продукта (string)
    'is_subscribed': np.random.choice([True, False], size=100)  # булевое значение подписки (bool)
}

# создание DataFrame
df = pd.DataFrame(data)

# 1. Применение Binarizer к income_usd с порогом = среднему значению
threshold = df['income_usd'].mean()
print(f"Среднее значение income_usd (порог для бинаризации): {threshold:.2f}")

binarizer = Binarizer(threshold=threshold)
df['income_usd_binarized'] = binarizer.fit_transform(df[['income_usd']])

# 2. Применение StandardScaler к age_years
scaler = StandardScaler()
df['age_years_standarded'] = scaler.fit_transform(df[['age_years']])

# 3. Применение LabelEncoder к is_subscribed
encoder = LabelEncoder()
df['is_subscribed_encoded'] = encoder.fit_transform(df['is_subscribed'])

# Проверка результатов
print("\n" + "="*70)
print("Результаты преобразований:\n")

print("1. Binarizer для income_usd:")
print(df[['income_usd', 'income_usd_binarized']].head(10))
print(f"\nРаспределение бинаризованных значений:")
print(df['income_usd_binarized'].value_counts())

print("\n" + "="*70)
print("2. StandardScaler для age_years:")
print(df[['age_years', 'age_years_standarded']].head(10))
print(f"\nСтатистика стандартизированных значений:")
print(f"Среднее: {df['age_years_standarded'].mean():.10f}")
print(f"Стандартное отклонение: {df['age_years_standarded'].std():.10f}")

print("\n" + "="*70)
print("3. LabelEncoder для is_subscribed:")
print(df[['is_subscribed', 'is_subscribed_encoded']].head(10))
print(f"\nСоответствие меток:")
print(f"False -> {encoder.transform([False])[0]}")
print(f"True -> {encoder.transform([True])[0]}")

### Задание 3
Примените преобразования к вашим данным, используя объект ColumnTransformer внутри Pipeline. Используйте преобразования из предыдущего задания:
Binarizer — к колонке income_usd (в качестве границы возьмите средние значения),
StandardScaler — к колонке age_years,
OneHotEncoder — к колонке is_subscribed.

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.preprocessing import Binarizer, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

np.random.seed(42)
np.random.default_rng(42)

# генерация данных для каждого столбца
data = {
    'temperature_celsius': np.random.uniform(20, 35, size=100),  # температура в градусах Цельсия (float)
    'age_years': np.random.randint(18, 65, size=100),  # возраст в годах (int)
    'timestamp_event': [pd.Timestamp('20230101') + timedelta(days=i) for i in range(100)],  # время события (datetime)
    'product_category': np.random.choice(['electronics', 'clothing', 'food'], size=100),  # категория продукта (string)
    'is_purchased': np.random.choice([True, False], size=100),  # булевое значение приобретения (bool)
    'humidity_percentage': np.random.uniform(40, 80, size=100),  # влажность в процентах (float)
    'income_usd': np.random.randint(20000, 100000, size=100),  # доход в долларах США (int)
    'last_updated': [pd.Timestamp('20240101') + timedelta(days=i) for i in range(100)],  # последнее обновление (datetime)
    'product_name': ['Product_' + str(i) for i in range(100)],  # название продукта (string)
    'is_subscribed': np.random.choice([True, False], size=100)  # булевое значение подписки (bool)
}
df = pd.DataFrame(data)

# Вычисляем среднее значение для Binarizer
threshold = df['income_usd'].mean()
print(f"Среднее значение income_usd (порог для бинаризации): {threshold:.2f}")
print("\n" + "="*70 + "\n")

# создание ColumnTransformer с преобразованиями для различных колонок
preprocessor = ColumnTransformer(
    transformers=[
        ('binarizer', Binarizer(threshold=threshold), ['income_usd']),
        ('scaler', StandardScaler(), ['age_years']),
        ('onehot', OneHotEncoder(sparse_output=False, drop=None), ['is_subscribed'])
    ],
    remainder='drop'  # остальные колонки НЕ включаем (избегаем проблем с типами)
)

# создание Pipeline с преобразованиями
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# применение преобразований
transformed_data = pipe.fit_transform(df)

print("Результаты преобразований:")
print(f"Форма преобразованных данных: {transformed_data.shape}")
print(f"Тип данных: {type(transformed_data)}")
print("\n" + "="*70 + "\n")

# Получаем имена признаков
feature_names = []
# Binarizer - 1 признак
feature_names.append('income_usd_binarized')
# StandardScaler - 1 признак
feature_names.append('age_years_scaled')
# OneHotEncoder - получаем имена автоматически
ohe_features = pipe.named_steps['preprocessor'].named_transformers_['onehot'].get_feature_names_out(['is_subscribed'])
feature_names.extend(ohe_features)

print(f"Имена признаков после преобразования: {feature_names}")
print("\n" + "="*70 + "\n")

# Создаём DataFrame с результатами
df_transformed = pd.DataFrame(
    transformed_data, 
    columns=feature_names
)

print("Первые 10 строк преобразованных данных:")
print(df_transformed.head(10))

print("\n" + "="*70)
print("\nСравнение исходных и преобразованных данных:")
comparison = pd.DataFrame({
    'income_usd': df['income_usd'].head(10).values,
    'income_binarized': df_transformed['income_usd_binarized'].head(10).values,
    'age_years': df['age_years'].head(10).values,
    'age_scaled': df_transformed['age_years_scaled'].head(10).values.round(3),
    'is_subscribed': df['is_subscribed'].head(10).values,
})
print(comparison)

print("\n" + "="*70)
print("\nСтатистика преобразованных признаков:")
print(f"\nBinarizer (income_usd):")
print(f"  Значений 0 (ниже среднего): {(df_transformed['income_usd_binarized'] == 0).sum()}")
print(f"  Значений 1 (выше среднего): {(df_transformed['income_usd_binarized'] == 1).sum()}")

print(f"\nStandardScaler (age_years):")
print(f"  Среднее: {df_transformed['age_years_scaled'].mean():.10f}")
print(f"  Std: {df_transformed['age_years_scaled'].std():.2f}")

print(f"\nOneHotEncoder (is_subscribed):")
print(f"  False: {df_transformed['is_subscribed_False'].sum():.0f}")
print(f"  True: {df_transformed['is_subscribed_True'].sum():.0f}")

print("\n" + "="*70)
print("\nПолная структура преобразованных данных:")
print(df_transformed.info())
print("\nПервые 5 строк:")
print(df_transformed.head())

### Задание 4
Вы работаете с данными о ежедневных температурах в градусах Цельсия за год в определённом регионе. Ваша задача — предобработать временные данные, чтобы затем их можно было использовать в модели машинного обучения. Выполните следующие шаги:
извлеките признаки из даты,
рассчитайте среднюю температуру за последние семь дней (скользящее окно) и накопительную сумму за весь период,
добавьте признаки, отражающие общий тренд в данных (сумма температур за каждый месяц) и периодичность событий (средняя температура по месяцам).

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# генерация случайных данных о температурах за год
np.random.seed(42)
np.random.default_rng(42)
start_date = pd.Timestamp('2023-01-01')
end_date = pd.Timestamp('2023-12-31')
dates = pd.date_range(start=start_date, end=end_date)
temperatures = np.random.uniform(low=-10.0, high=30.0, size=len(dates))
temperature_data = pd.DataFrame({'Date': dates, 'Temperature_Celsius': temperatures})

# ваш код для предобработки временных признаков #

# 1. Извлечение признаков из даты
temperature_data['Month'] = temperature_data['Date'].dt.month
temperature_data['Weekday'] = temperature_data['Date'].dt.weekday
temperature_data['Hour'] = temperature_data['Date'].dt.hour

# 2. Скользящие окна и накопительные статистики
temperature_data['Cumulative_Sum'] = temperature_data['Temperature_Celsius'].cumsum()

# 3. Периодичность и тренды
temperature_data['Monthly_Sum'] = temperature_data.groupby('Month')['Temperature_Celsius'].transform('sum')
temperature_data['Monthly_Mean'] = temperature_data.groupby('Month')['Temperature_Celsius'].transform('mean')

# вывод обработанных данных
print(temperature_data.head())